# Multi Anno Merged File Batch Processor

Created on Mon Dec 5 11:37:53 2022

@author: nilesh@4basecare.com

In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import os
import shutil
from pathlib import Path
import pandas as pd
import numpy as np
import csv
from tqdm import tqdm
import warnings

# To ignore all warnings
warnings.filterwarnings("ignore")

In [2]:
# For converting csv file to tsv file
# Checking indir, if the folder exists and contains "_merged_output.csv"
folder_path = r"E:\indigene"

def folder_contains_csv(folder_path):
    if not os.path.exists(folder_path) or not os.path.isdir(folder_path):
        print(f"The provided path '{folder_path}' is not a valid folder.")
        return False

    # List all files in the folder
    files = os.listdir(folder_path)

    # Check if any of the files have a ".csv" extension
    for file in files:
        if file.lower().endswith('_filtered.csv'):
            return True

    return False



# Specify the input directory containing CSV files and the output directory for TSV files
input_directory = folder_path
output_directory = r"E:\indigene\tsv"

# Ensure the output directory exists, create it if necessary
os.makedirs(output_directory, exist_ok=True)

# Get a list of CSV files in the input directory
tsv_files = [filename for filename in os.listdir(input_directory) if filename.endswith('.csv')]

# Initialize the progress bar
with tqdm(total=len(tsv_files), desc="Converting") as pbar:
    # Iterate through all CSV files in the input directory
    for filename in tsv_files:
        # Construct the full paths for input and output files
        input_csv_file = os.path.join(input_directory, filename)
        output_tsv_file = os.path.join(output_directory, filename[:-4] + '.tsv')

        # Open the CSV file for reading and the TSV file for writing
        with open(input_csv_file, 'r') as csv_file, open(output_tsv_file, 'w', newline='') as tsv_file:
            # Create a CSV reader and TSV writer
            csv_reader = csv.reader(csv_file)
            tsv_writer = csv.writer(tsv_file, delimiter='\t')

            # Iterate through each row in the CSV file and write it to the TSV file
            for row in csv_reader:
                tsv_writer.writerow(row)

        # Update the progress bar
        pbar.update(1)

# Finished converting all CSV files to TSV files in the directory
print("Conversion complete.")

Converting: 100%|██████████| 2553/2553 [00:15<00:00, 165.51it/s]

Conversion complete.


In [3]:
# I/O and Chunk Settings

## Folder containing *TSV* MultiAnno Merged Output of Sample's VCF
dirpath = r"E:\indigene\tsv"

## Folder Path for Saving the Chunks
savepath = r"E:\indigene\combine"

## Sample List
sample_list = r"E:\indigene\tsv\indigene_samp.txt"

# Path for Genes file for Gene based filtration
# genes = pd.read_csv("/home/bioinfo/Nilesh/HRRdb_Samples/Scripts/HRR_genes.txt", sep='/t')
#genes = ["MSH2","MSH3","MSH5","MSH6","MLH1","MLH2","MLH3","MLH4"]
#genes_series = pd.Series(genes, name="MMR Genes")

# Set Number of files per chunk.
## If total files is 5, and chunk size is 3,
## then 2 folders will be made,
## 1st folder will have 3 files, 2nd one will have 2 files
chunk_size = 700

# Sample File listing
tsv_files = [filename for filename in os.listdir(dirpath) if filename.endswith('_filtered.tsv')]

# Chunk List creation based on  Chunk Size
chunked_list = [tsv_files[i:i+chunk_size] for i in range(0, len(tsv_files), chunk_size)]

In [4]:
dt=dict(enumerate(chunked_list))  #Nested List converted to Dictionary

dt_list = [[k,v] for k, values in dt.items() for v in values]

# dt_list = []                     #For Loop Expansion of above List Comprehension for Understanding
# for keys, values in dt.items():  #for making file list per chunk used later for copying
#   for value in values:
#       dt_list.append([keys, value])



In [6]:
for folder_index, file_name in dt_list:
    source_path = f"{dirpath}\\{file_name}"
    destination_path = f"{savepath}\\Folder{folder_index}"

    # Create the destination folder if it doesn't exist
    os.makedirs(destination_path, exist_ok=True)

    # Copy the file
    shutil.copy(source_path, destination_path)

In [7]:
# Standard Filtration
for keys in tqdm(dt, desc="Processing folders"):
    folder_path = f"{savepath}\\Folder{keys}"
    print("####")

    os.chdir(folder_path)
    key_files = sorted(Path('./').glob('*.tsv'))
    #print(key_files)


    samples_list = []  # Create a list to store DataFrames

    for file in tqdm(key_files, desc=f"Processing files in Folder{keys}", leave=False):
        samp_name = file.name.replace("_filtered.tsv", "")
        sel_cols = ['CHROM_x', 'POS_x', 'End_x', 'REF_x', 'ALT_x', 'Ref.Gene', 'Func.ensGene', 'ExonicFunc.ensGene', 'AAChange.ensGene', 'Interpro_domain', 'avsnp150', 'CLNDN', 'CLNDISDB', 'clinvar: Clinvar ']  # f'{samp_name}:GT'
        file_df = pd.read_csv(file, sep='\t', usecols= sel_cols, low_memory=False)
        file_df['SID'] = samp_name
        file_df['Chr_SERA'] = file_df['CHROM_x'].astype(str) + ' | ' + file_df['POS_x'].astype(str) + ' | ' + file_df['End_x'].astype(str) + ' | ' + file_df['REF_x'].astype(str) + ' | ' + file_df['ALT_x'].astype(str)

        # List of columns to be removed
        static_columns_to_remove = ['CHROM_x', 'POS_x', 'End_x', 'REF_x', 'ALT_x']

        # Drop the specified columns
        file_df.drop(columns= static_columns_to_remove, inplace=True)

        # Specify the desired order of columns
        desired_columns_order = ['SID', 'Chr_SERA'] + [col for col in file_df.columns if col not in ['SID', 'Chr_SERA']]
        # Reorder the columns
        file_df = file_df[desired_columns_order]

        samples_list.append(file_df)  # Append each DataFrame to the list
        samples = pd.concat(samples_list, ignore_index=True)  # Concatenate the list of DataFrames into one

        samples.to_csv('./'+str(keys)+'_df.tsv', index = False,  sep='\t')
    os.chdir('..')

Processing folders:   0%|          | 0/4 [00:00<?, ?it/s]

####


Processing folders:  25%|██▌       | 1/4 [00:36<01:49, 36.62s/it]

####


Processing folders:  50%|█████     | 2/4 [01:01<00:58, 29.47s/it]

####


Processing folders:  75%|███████▌  | 3/4 [01:31<00:29, 29.89s/it]

####


Processing folders: 100%|██████████| 4/4 [01:47<00:00, 26.78s/it]


In [8]:
# List of TSV file names
#filtered_files = sorted(Path(savepath).rglob('*_gene_df.tsv')) #Gene Filtration
filtered_files = sorted(Path(savepath).rglob('*_df.tsv')) #Standard Filtration
print(filtered_files)
# Read TSV files into a list of DataFrames
filt_df = [pd.read_csv(ff, sep='\t') for ff in filtered_files]

# Append DataFrames vertically
appended_df = pd.concat(filt_df, ignore_index=True)

# Write the appended DataFrame to a new TSV file
appended_df.to_csv(savepath+'\\'+'appended_file.tsv', sep='\t', index=False)


[WindowsPath('E:/indigene/combine/Folder0/0_df.tsv'), WindowsPath('E:/indigene/combine/Folder1/1_df.tsv'), WindowsPath('E:/indigene/combine/Folder2/2_df.tsv'), WindowsPath('E:/indigene/combine/Folder3/3_df.tsv')]


In [9]:
# Appended File converted to Excel file

# Create a new DataFrame for the Sample List
df_sample_list = pd.DataFrame(columns=['Original Sample List', 'Samples after Filtration'])

# Read values from text file for the first column
with open(sample_list, 'r') as file: #, encoding='utf-16-le'
    original_samples = file.read().splitlines()

# Get unique values from the 'SID' column of appended_df for the second column
filtered_samples = appended_df['SID'].unique().tolist()

# Pad the shorter list with NaN values to match the length of the longer list
max_length = max(len(original_samples), len(filtered_samples))
original_samples += [np.nan] * (max_length - len(original_samples))
filtered_samples += [np.nan] * (max_length - len(filtered_samples))

# Create DataFrame with columns of different lengths
df_sample_list = pd.DataFrame({'Original Sample List': original_samples, 'Samples after Filtration': filtered_samples})

#Create a new Excel writer object
with pd.ExcelWriter('output_Filtered.xlsx') as writer:
    # Write the TSV data to the first sheet (Filtered File)
    appended_df.to_excel(writer, sheet_name='Filtered File', index=False)

    # Write the Sample List DataFrame to the second sheet
    df_sample_list.to_excel(writer, sheet_name='Sample List', index=False)